In [0]:
# Assuming you have already set up your Spark session
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("Read Table ") \
    .getOrCreate()

# Replace 'your_database' and 'your_table' with your actual database and table names
database_name = "default"
table_name = "data_forbes"

# Read the table
df = spark.table(f"{database_name}.{table_name}")

# Show the data
df.show(5)


+--------------------+----------+-------------------+-------------+-------------+-----------+-----+-------+------+----+--------------------+
|             company|    sector|           industry|    continent|      country|marketvalue|sales|profits|assets|rank|       forbeswebpage|
+--------------------+----------+-------------------+-------------+-------------+-----------+-----+-------+------+----+--------------------+
|                ICBC|Financials|        Major Banks|         Asia|        China|      215.6|148.7|   42.7|3124.9|   1|http://www.forbes...|
|China Constructio...|Financials|     Regional Banks|         Asia|        China|      174.4|121.3|   34.2|2449.5|   2|http://www.forbes...|
|Agricultural Bank...|Financials|     Regional Banks|         Asia|        China|      141.1|136.4|   27.0|2405.4|   3|http://www.forbes...|
|      JPMorgan Chase|Financials|        Major Banks|North America|United States|      229.7|105.7|   17.3|2435.3|   4|http://www.forbes...|
|  Berkshire 

In [0]:
from pyspark.sql.functions import *
# Adding a new column 'NewValue' based on 'Value'
df_tr = df.withColumn("Costs", col("sales") - col("profits"))

# Show the new DataFrame
df_tr.show(5)

+--------------------+----------+-------------------+-------------+-------------+-----------+-----+-------+------+----+--------------------+----------+
|             company|    sector|           industry|    continent|      country|marketvalue|sales|profits|assets|rank|       forbeswebpage|     Costs|
+--------------------+----------+-------------------+-------------+-------------+-----------+-----+-------+------+----+--------------------+----------+
|                ICBC|Financials|        Major Banks|         Asia|        China|      215.6|148.7|   42.7|3124.9|   1|http://www.forbes...|     106.0|
|China Constructio...|Financials|     Regional Banks|         Asia|        China|      174.4|121.3|   34.2|2449.5|   2|http://www.forbes...| 87.100006|
|Agricultural Bank...|Financials|     Regional Banks|         Asia|        China|      141.1|136.4|   27.0|2405.4|   3|http://www.forbes...|109.399994|
|      JPMorgan Chase|Financials|        Major Banks|North America|United States|      2

In [0]:
# Unpersisting the DataFrame
df_tr.unpersist()

# Optionally, delete the variable
del df_tr

In [0]:
df.columns

Out[7]: ['company',
 'sector',
 'industry',
 'continent',
 'country',
 'marketvalue',
 'sales',
 'profits',
 'assets',
 'rank',
 'forbeswebpage']

In [0]:
df_tr = df.select('company','marketvalue','sales','profits',(col("sales") - col("profits")).alias('cost'))

df_tr.show(5)

+--------------------+-----------+-----+-------+----------+
|             company|marketvalue|sales|profits|      cost|
+--------------------+-----------+-----+-------+----------+
|                ICBC|      215.6|148.7|   42.7|     106.0|
|China Constructio...|      174.4|121.3|   34.2| 87.100006|
|Agricultural Bank...|      141.1|136.4|   27.0|109.399994|
|      JPMorgan Chase|      229.7|105.7|   17.3| 88.399994|
|  Berkshire Hathaway|      309.1|178.8|   19.5|     159.3|
+--------------------+-----------+-----+-------+----------+
only showing top 5 rows



In [0]:
df_tr.select('*').where(df_tr.profits > 20).show()

+--------------------+-----------+-----+-------+----------+
|             company|marketvalue|sales|profits|      cost|
+--------------------+-----------+-----+-------+----------+
|                ICBC|      215.6|148.7|   42.7|     106.0|
|China Constructio...|      174.4|121.3|   34.2| 87.100006|
|Agricultural Bank...|      141.1|136.4|   27.0|109.399994|
|         Exxon Mobil|      422.3|394.0|   32.6|     361.4|
|         Wells Fargo|      261.4| 88.7|   21.9| 66.799995|
|       Bank of China|      124.2|105.1|   25.5|      79.6|
|          PetroChina|      202.0|328.5|   21.1|     307.4|
|               Apple|      483.1|173.8|   37.0|     136.8|
|                  BP|      148.8|379.2|   23.6|     355.6|
|             Chevron|      227.2|211.8|   21.4| 190.40001|
|             Gazprom|       88.8|164.6|   39.0|125.600006|
| Samsung Electronics|      186.5|208.9|   27.2|     181.7|
|           Microsoft|      343.8| 83.3|   22.8| 60.500004|
|            Vodafone|       96.9| 65.1|

In [0]:
from pyspark.sql import Window
wndw = Window.orderBy(col("Cost").desc())
df_tr = df_tr.withColumn('cost_rank',rank().over(wndw))

In [0]:
top_high_cost = df_tr.filter(df_tr.cost_rank <= 10).orderBy(df_tr.cost_rank.asc())
top_high_cost.show()

+--------------------+-----------+-----+-------+-----+---------+
|             company|marketvalue|sales|profits| cost|cost_rank|
+--------------------+-----------+-----+-------+-----+---------+
|     Wal-Mart Stores|      247.9|476.5|   16.0|460.5|        1|
|   Royal Dutch Shell|      234.1|451.4|   16.4|435.0|        2|
|Sinopec-China Pet...|       94.7|445.3|   10.9|434.4|        3|
|         Exxon Mobil|      422.3|394.0|   32.6|361.4|        4|
|                  BP|      148.8|379.2|   23.6|355.6|        5|
|          PetroChina|      202.0|328.5|   21.1|307.4|        6|
|    Volkswagen Group|      119.0|261.5|   12.0|249.5|        7|
|Glencore Internat...|       67.5|232.6|   -7.5|240.1|        8|
|        Toyota Motor|      193.5|255.6|   18.8|236.8|        9|
|               Total|      149.8|227.9|   11.2|216.7|       10|
+--------------------+-----------+-----+-------+-----+---------+



In [0]:
top_high_cost.write.csv('high_cost_operating_companies.csv')